<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/intervviewZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0jhk54l0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0jhk54l0
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [41]:
!pip install moviepy

In [42]:
!pip install ffmpeg-python

In [43]:
import whisper
import ffmpeg
import numpy as np
import json, re, os
import glob
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip

In [44]:
!cp ./drive/MyDrive/IZ-2218確認３/*.* ./

In [45]:
def is_webm(path):
    with open('./' + path) as f:
        for _line in f:
            if _line.rstrip()[-5:] == '.webm':
                return True
    return False

In [46]:
def conv_webm(input_file):
    #!ffmpeg -allowed_extensions ALL -f hls -i ./drive/MyDrive/IZ-2218確認３/ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.m3u8 -c:v copy -y ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm
    webm_tmp = "./video_tmp.webm"
    try:
        (
            ffmpeg
            .input('./' + input_file, allowed_extensions='ALL')
            .output(webm_tmp, vcodec="copy")
            .overwrite_output()
            .run(capture_stderr=True, capture_stdout=True)
        )
        _clip = VideoFileClip(webm_tmp)
    except ffmpeg.Error as e:
        #print(f"{e.stderr.decode()}")
        #print(f"標準出力: {e.stdout.decode()}")
        _clip = None
    return _clip

In [47]:
def get_video_list(path):
    video_path = f'{path}*_video.m3u8'
    video_paths = glob.glob(video_path)
    video_paths.sort()
    return video_paths
def get_audio_list(path):
    audio_path = f'{path}*_audio.m3u8'
    audio_paths = glob.glob(audio_path)
    audio_paths.sort()
    return audio_paths

In [48]:
path = './'
video_paths = get_video_list(path)
audio_paths = get_audio_list(path)

In [49]:
video = []
for video_path in video_paths:
    _video = conv_webm(video_path) if is_webm(video_path) else VideoFileClip(video_path)
    video.append(_video)
print(video)

[<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7be7c98c35b0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7be6d29a4fa0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7be778b2b6a0>]


In [54]:
audio = []
wav_paths = []
for audio_path in audio_paths:
    wav_path = '.'.join(audio_path.split('.')[:-1]) + '.wav'
    audio_tmp = AudioFileClip(audio_path)
    audio_tmp.write_audiofile(wav_path, ffmpeg_params=["-ac", "1"])
    audio.append(audio_tmp)
    wav_paths.append(wav_path)
print(audio)

MoviePy - Writing audio in ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_369679075__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_462504175__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_audio.wav


MoviePy - Done.
[<moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7be6d2a305e0>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7be6c0287130>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7be6c9b7cd60>]


In [51]:
model = whisper.load_model("large")

100%|██████████████████████████████████████| 2.88G/2.88G [00:19<00:00, 155MiB/s]
  checkpoint = torch.load(fp, map_location=device)



In [56]:
asr_result = []
for wav_patn in wav_paths:
    print(f'---- {wav_patn} ----')
    _result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True)
    asr_result.append(_result)

---- ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_369679075__uid_e_audio.wav ----
[00:19.360 --> 00:21.520] お疲れ様です
[00:21.520 --> 00:24.320] 今日はいつも
[00:24.320 --> 00:26.160] いつもじゃないんですけれども
[00:26.160 --> 00:28.360] 今日もちょっと30分ぐらいの
[00:28.360 --> 00:31.960] インタビューをさせていただきたいなと思います
[00:34.500 --> 00:39.020] 今日のタイトルに関しては
[00:39.020 --> 00:40.380] 少々お待ちください
[00:41.000 --> 00:42.360] 今出します
[00:53.200 --> 00:55.200] 上平さんは
[00:55.200 --> 00:57.140] ホットペーパーデューティーを
[00:57.140 --> 00:58.260] 使っていらっしゃいます
[00:58.260 --> 00:58.560] いらっしゃいますか
[01:00.600 --> 01:01.200] はい
[01:02.220 --> 01:03.500] それでは少し
[01:04.260 --> 01:07.140] 質問をさせていただきます
[01:23.820 --> 01:25.860] 上平さんすみません
[01:25.860 --> 01:26.720] 質問します
[01:28.260 --> 01:29.520] ホットペーパーデューティーで
[01:29.520 --> 01:31.980] 予約したことのあるジャンルを
[01:31.980 --> 01:32.980] 教えてください
[01:34.000 --> 01:35.900] これから複数
[01:35.900 --> 01:38.100] 候補を読み上げます
[01:38.760 --> 01:39.360] そして
[01:39.360 --> 01:41.920] はいと言えて答えてください
[01:44.160 --> 01:45.760] 部屋の
[0

In [ ]:
whisper_result = {}
whisper_segment = []
for asr in asr_result['segments']:
    if asr['end'] - asr['start'] >= 0.6 and len(asr['text']) >= 2:
        asr['text'] += '。'
        whisper_segment.append(asr)
whisper_result['segments'] = whisper_segment

In [ ]:
whisper_result

In [ ]:
#speaker_count = 2
speaker_count = 3

import torch
pipeline.to(torch.device("cuda"))
diarization_result = pipeline(audio_path, num_speakers=speaker_count)
#diarization_result = pipeline(audio_path, min_speakers=2, max_speakers=5)

In [ ]:
diarization_result

In [ ]:
#final_result = diarize_text(asr_result, diarization_result)
final_result = diarize_text(whisper_result, diarization_result)

In [ ]:
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
INT_COUNT = []
for seg, spk, sent in final_result:
  if spk is None:
    spk = INTERVIEW[-1]['spk']
  count = len(prog.findall(sent))
  spk_no = int(spk.split('_')[-1])
  if count > 0: print(str(sent))
  _seg = {'start': round(seg.start, 2), 'end': round(seg.end, 2), 'text': sent, 'count': count, 'spk': spk, 'spk_no': spk_no}
  INTERVIEW.append(_seg)
  INT_COUNT.append(int(spk[-1]))

speaker_count = max(INT_COUNT) + 1

speek = [0] * speaker_count
for INT in INTERVIEW:
    speek[INT["spk_no"]] += INT["count"]
inteviewer_index = np.argmax(speek)
monitor_index = np.argmin(speek)

print(speek)
print(f'インタービューア ： SPEAKER_0{inteviewer_index}')
print(f'モニター　　　　 ： SPEAKER_0{monitor_index}')

In [ ]:
for seg, spk, sent in final_result:
    speeker_name = spk
    #speeker_name =   'モニタ　　　　'
    #if spk == 'SPEAKER_0' + str(inteviewer_index):
    #  speeker_name = 'インタビューア'
    line = f'{seg.start:.2f}s~{seg.end:.2f}s {speeker_name} {sent}'
    print(line)

In [ ]:
InterReview_json = {"movie_name": video_name}
speak_json = []
for seg, spk, sent in final_result:
    _speak = {"start": f'{seg.start:.2f}s', "end": f'{seg.end:.2f}s', 'speaker': spk, 'speak': sent}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

json_path = '.'.join(video_path.split('.')[:-1]) + '.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)


In [ ]:
!ls -alh ./

In [ ]:
!cp ./InterviewZero_0_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_1_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_2_audio_mask.json ./drive/MyDrive